<center><img src="images/header.png"></center>

<h1><center>Лекция №4: Введение в машинное обучение</center></h1>
<hr>
<h2><center>Методы обучения без учителя: Кластеризация</center></h2>
<h3><center>Шестаков Андрей</center></h3>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print(u'Так надо')

# Методы обучение без учителя (Unsupervised)

* В чем отличие от Supervised методов?

* Кластеризация
* Уменьшение размерности
    * Метод главных компонент
    * Многомерное шкалирование
    * Тематические модели*
* Поиск ассоциативных правил

# Кластеризация

Основная задача кластерного анализа — разбиение исходного набора объектов на группы (кластеры) таким образом, чтобы объекты в группе были похожи друг на друга, а объекты из разных групп - отличались.

<center><img src="https://i.ytimg.com/vi/zPJtDohab-g/maxresdefault.jpg" width=600></center>

## Цели кластерного анализа

* **Поиск структуры** в данных и ее **интерпретация**
* Поиск аномальных объектов
* Детальный анализ отдельных кластеров
* Формирование признаков на основе кластеризации

## Группы методов

* Методы основанные на прототипах
    * Каждый кластер ассоциируется с виртуальрным "эталонным" объектом
* Иерархические методы
    * Не простое разбиение на целая иерархия
* Плотностные методы
    * Ищем плотные скопления точек в признаковом пространстве
* Вероятностные методы
    * Предполагаем, что данные порождены некоторой смесью вероятностных распределений
* Спектральные методы
    * Испольюзуем замечательные спектральные свойства разных матриц
* Сеточные методы
    * Бьем признаковое пространство на сегменты
* ...

## Алгоритм k-means
### Неформальное [демо](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

## Алгоритм k-means

* Дано множество объектов $X = \{x_1, x_2, \dots, x_N\}$
* Кластер $C_k \Leftrightarrow \text{ центройд } \mu_k$
* Объект $x_i \in C_k \Leftrightarrow \mu_k = \arg \min\limits_{\mu_j} \|x_i - \mu_j \|^2$
* Надо найти такое разбиение на $K$ кластеров, чтобы минизировать
$$ L(C) = \sum_{k=1}^K\sum_{i\in C_k} ||x_i - \mu_k||^2 \rightarrow \min\limits_C $$
$$\mu_k = \frac{1}{|C_k|} \sum _{x_n \in C_k} x_n$$

* Метод $k$-средних является итеративным алгоритмом разбиения множества объектов на $K$ кластеров 

## Алгоритм k-means
1. Выбрать $K$ начальных центроидов случайным образом  $\rightarrow \mu_k, \ k=1\dots K$
2. Для каждой точки из датасета присвоить кластер, соответствующий ближайшему центроиду
$$C_k = \{x_n : ||x_n - \mu_k||^2 \leq ||x_n - \mu_l||^2 \quad \forall l \neq k \} $$
3. Обновить центройды: 
$$\mu_k = \frac{1}{|C_k|} \sum _{x_n \in C_k} x_n$$
4. Повторять 2 и 3 до тех пор, пока изменения перестанут быть существенными 


<center><img src='images/Kmeans_animation.gif' width=500></center>

# Основные факторы
* Начальная инициализация центройдов
* Количество кластеров

## Kак выбрать K?

* Не пользоваться обычным k-means (X-means, ik-means)
* Посмотреть на меры качества кластеризации
* Воспользоваться эвристиками

## Elbow method (Метод локтя)

* Критерий минимизации k-means
$$ L(C) = \sum_{k=1}^K\sum_{i\in C_k} ||x_i - \mu_k||^2 \rightarrow \min\limits_C $$
* Давайте возьмем всевозможные $K$, для каждого запустим алгоритм, посчитаем на результате $L(C)$ и выберем минимум!

* Ничего не выйдет... Почему?

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1) 


crit = []

for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=1).fit(X)
    crit.append(np.sqrt(kmeans.inertia_))
    
def elbow_demo(k=2):
    
    X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1) 
    
    kmeans = KMeans(n_clusters=k, random_state=1).fit(X)
    
    fig, ax = plt.subplots(1,2)
    
    ax[0].scatter(X[:,0], X[:,1], c=kmeans.labels_)
    
    ax[0].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                  marker='o', c="white", alpha=1, s=200)
    
    ax[0].set_xlabel('$x_1$')
    ax[0].set_ylabel('$x_2$')

    for i, c in enumerate(kmeans.cluster_centers_):
        ax[0].scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)
        
    ax[1].plot(range(2,8), crit, marker='s')
    
    ax[1].set_xlabel('$k$')
    ax[1].set_ylabel('$L^{(k)}(C)$')
    
    plt.show()
    
    

## Elbow method (Метод локтя)

* Выбирают такое $k$, после которого функционал $L(C)$ уменьшается не слишком быстро
* Чуть более формально:
$$ D(k) = \frac{|L^{(k)}(C) - L^{(k+1)}(C)|}{|L^{(k-1)}(C) - L^{(k)}(C)|} \quad \text{"невелико"} $$

In [ ]:
plot = interact(elbow_demo, k=IntSlider(min=2,max=8,step=1,value=2))

## Важно!
* Эвристика и меры качества клатеризации носят лишь рекомендательный характер!
* Если они ничего не дают, то лучше ориентироваться на свои знания в предметной области
* Или "выжать" из полученной кластеризации максимум
    * *3 из 5 полученных кластеров интерпретируются - и то хорошо*

## Начальная инициализация центройдов
* Выбрать координаты $K$ случайных объектов из датасета
    * Производить случайные запуски много раз и выбрать наиболее оптимальную инициализацию
* Использовать результат другой кластеризации на $K$ кластеров
* k-means++

### K-means++
* Первый центройд выбираем случайным образом из объектов датасета
* Для каждой точки рассчитываем расстояние $d_{\min}(x_i) = \min_{\mu_j} \|x_i - \mu_j\|^2$
* Точка назначается следующим центройдом с вероятностью $p(x_i) \propto d_{\min}(x_i)$

In [ ]:
from sklearn.metrics import pairwise_distances

def demo_kmpp(iters=1):

    X, y = make_blobs(n_samples=550, cluster_std=1.5, n_features=2, centers=5, random_state=12345)

    X_grid1, X_grid2 = np.meshgrid(np.linspace(-12, 18, 500),
                                   np.linspace(-11, 8, 500))

    XX = np.c_[X_grid1.flatten(), X_grid2.flatten()]
    np.random.seed(1)
    centroids = np.empty((0, 2))

    for i in range(iters):
        if i == 0:
            d = np.ones_like(y, dtype=float)
        else:
            d = pairwise_distances(X, centroids, metric='euclidean').min(axis=1)
        weights = d/d.sum()

        centroid_idx = np.random.choice(X.shape[0], size=1, replace=False, p=weights)[0]
        centroids = np.r_[centroids, X[centroid_idx, np.newaxis]]

    d_grid = pairwise_distances(XX, centroids, metric='euclidean').min(axis=1)

    d_grid = d_grid.reshape(X_grid1.shape)
    d_grid = d_grid/d_grid.max()

    levels = np.linspace(0, 1, 100)

    plt.contourf(X_grid1, X_grid2, d_grid, cmap=plt.cm.Blues, alpha=0.7, levels=levels)
    plt.scatter(X[:, 0], X[:, 1], s=100)

    centers = centroids
    
    plt.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=500, edgecolor='k')

    for i, c in enumerate(centers):
        plt.scatter(c[0], c[1], marker='$%d$' % (i+1), alpha=1,
                    s=100, edgecolor='k')

    plt.xlabel('$x_1$', fontsize=15)
    plt.ylabel('$x_2$', fontsize=15)

    plt.tight_layout()
    plt.axis('equal')
    plt.show()


In [ ]:
interact(demo_kmpp, iters=IntSlider(min=1,max=6,step=1,value=1))

## Резюме

* Метод k-средних – жадный итеративный алгоритм
* Зависит от начальных центройдов и их количества

#### Преимущества
* Прост как пробка
* Имеет множество модификаций
* Интерпретация кластеров через центройды

#### Недостатки

* Подразумевает выпуклые кластеры
<center><img src='images/kmeans_2moons.png' width=800></center>

* Всегда* на выходе будет k кластеров
<center><img src='images/kmeans_digits.png' width=800></center>

<center><img src='images/kmeans-guys.jpeg'></center>


# Спектральная кластеризация

## Небольшой спойлер к лекцям по сетям

**Граф** - математическая модель представления сетевых структур.

Графом $\mathcal{G}$ называется пара множеств $\mathcal{G} = (V, E)$, где $V$ содержит в себе множество *вершин (узлов, vertices, nodes)*, а $E$ содержит множество *ребер (связей, edges, links)*. <br/> 

<center><img src='images/clique_init.png' width=800></center>


## Матрица смежности, Incidence matrix, Association matrix.

Матрица смежности - это квадратная матрица размера $n \times n$. Обозначим ее буквой $\mathbf{A}$ <br/>

**В невзвешенном графе:**<br/>
$\mathbf{A_{ij}} = 1$, если в графе есть ребро между вершинами $v_i$ и $v_j$ <br/>
$\mathbf{A_{ij}} = 0$, иначе

**Во взвешенном графе:**<br/>
$\mathbf{A_{ij}} = w_{ij}$, если в графе есть ребро между вершинами $v_i$ и $v_j$ <br/>
$\mathbf{A_{ij}} = 0$, иначе

Для графа выше матрица смежности имеет следующий вид:

<center><img src='images/clique_matrix.png'></center>

In [ ]:
A = \
np.array([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
          [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
          [0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
          [0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
          [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
          [0, 0, 0, 0, 0, 1, 0, 1, 1, 0]])

d = A.sum(axis=1) # Степени вершин (количество соседей)
d

In [ ]:
D = np.diag(d)
D

## Компоненты связности и Лапласиан
* На время представим, что в графе отсутствуют ребра $e_{3,7}$, $e_{2,8}$, $e_{6,10}$
<center><img src='images/clique_init.png' width=800></center>
* Тогда в графе возникают 2 связные компоненты $\{v_1,v_2,v_3,v_4,v_5,v_6\}$ и $\{v_7,v_8,v_9,v_{10}\}$
* Рассмотрим матрицу $L = D-A$ - Лапласиан

In [ ]:
A = \
np.array([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 1, 0, 1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
          [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
          [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]])
d = A.sum(axis=1) 
D = np.diag(d)
L = D-A
L

## Замечательные свойства Лапласиана

* Симметричная и положительно определенная матрица
$$ \forall f: f^\top L f = f^\top D f -  f^\top A f = \frac{1}{2} \left(\sum_{i=1}^nd_i f^2_i - 2\sum_{i,j=1}^nf_if_ja_{ij} + \sum_{j=1}^nd_j f^2_j\right) = \frac{1}{2} a_{ij}(f_i-f_j)^2 \geq 0$$
* Все собственные числа $\lambda_1\leq\dots\leq\lambda_n$ неотрицательные 
* Наименьшее собственное число $\lambda_1=0$ с единичным собственныем вектором
* Если в графе с матрицей смежности $A$ есть $k$ компонент, то кратность собственного числа $\lambda_1=0$ равна $k$, а соответствующие собственные вектора задают разбиение на компоненты

In [ ]:
eig_vals, eig_vects = np.linalg.eig(L)
idx = np.argsort(eig_vals) # отсортируем по возрастанию
eig_vects = eig_vects[:,idx] # так же пересортируем

print(eig_vects[:,0])
print(eig_vects[:,1])

In [ ]:
# Вернемся в неблагоприятный кейс:
# вернем недостающие ребра и повторим расчеты
A = \
np.array([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
          [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
          [0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
          [0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
          [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
          [0, 0, 0, 0, 0, 1, 0, 1, 1, 0]])
d = A.sum(axis=1) 
D = np.diag(d)
L = D-A
L

In [ ]:
eig_vals, eig_vects = np.linalg.eig(L)
idx = np.argsort(eig_vals) # отсортируем по возрастанию
eig_vects = eig_vects[:,idx] # так же пересортируем

Y = eig_vects[:,:2] # Возьмем 2 наименьших собственных вектора j
# 2 - потому что хотим разделить на 2 части

In [ ]:
eig_vals

In [ ]:
Y

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1])

А это уже можно кластеризовать с помощью k-means и получить разбиение графа!

## Спектральная кластеризация для табличных данных

* Имея на входе исходные данные (объекты по строкам, признаки по столбцам) надо получить что-то наподобии матрицы смежности $A$
* Если объекты $x_i$ и $x_j$ "похожи", то $a_{ij} \rightarrow 1$
* Можно например использовать функцию 
$$A_{ij} = K(x_i,x_j) = \exp\left(\frac{-\|x_i-x_j\|^2}{2\sigma^2}\right)$$

## Aлгоритм
**Вход:**
* Данные $X\in\mathbb{R}^{n\times m}$
* Kоличество кластеров $k$
* Ширина окна $\sigma$

**Шаги**
1. Расчитываем афинитивность между всеми парами объектов - получаем матрицу $A$ $$A_{ij} = \exp\left(\frac{-\|x_i-x_j\|^2}{2\sigma^2}\right)$$
2. Считаем лапласиан $L=D-A$
3. Находим $k$ наименьших собственных векторов матрицы $L$ - Формируем матрицу $Y\in \mathbb{R}^{n\times k}$
4. Кластеризуем на $k$ кластеров с помощью метода $k$-средних

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(random_state=123, noise=0.1)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
from sklearn.cluster import KMeans, SpectralClustering

# model = KMeans(n_clusters=2)
model = SpectralClustering(n_clusters=2, gamma=30, random_state=123)

model.fit(X)
labels = model.labels_

plt.scatter(X[:,0], X[:,1], c=labels)

# Оценка качества кластеризаци

### Оценка качества кластеризации при известном groud truth

Пусть $\hat{\pi}$ - это полученное разбиение на кластеры, а $\pi^*$ - ground truth. 

<center><img src='http://cse3521.artifice.cc/images/iris-true-class-sepal.png' width=800></center>


#### Adjusted Rand Index

$$ \text{Rand}(\hat{\pi},\pi^*) = \frac{a + d}{a + b + c + d} \text{,}$$
где 
* $a$ количество пар объектов, находящихся в одинаковых кластерах в $\hat{\pi}$ и
$\pi^*$, 
* $b$ ($c$) количество пар объектов в одном и том же кластере в  $\hat{\pi}$ ($\pi^*$), но в разных в  $\pi^*$ ($\hat{\pi}$)
* $d$ количество пар объектов в разных кластерах в $\hat{\pi}$ и $\pi^*$

<center><img src='images/rand1.png' width=700></center>

#### Индекс Жаккара
$$ Jac(\hat{\pi}, \pi^*) = \frac{a}{a + b + c}$$

#### Точность, полнота и F-мера
$$ Precision(\hat{\pi}, \pi^*) = \frac{a}{a + b} $$
$$ Recall(\hat{\pi}, \pi^*) = \frac{a}{a + c} $$
$$ F-measure(\hat{\pi}, \pi^*) = \frac{2Precision \cdot Recall}{Precision + Recall} $$

## Меры валидности кластеров (no ground truth)

* Измеряют полученое разбиения по отношению к качествам хорошей кластеризации
    * Компактность объектов внутри кластера
    * Разделимость кластеров друг от друга
    
Про различные меры качества кластризации и меры валидности кластеров в sklearn можно почитать [тут](https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation) 

##### Критерий Silhouette

Пусть дана кластеризация в $K$ кластеров, и объект $i$ попал в $C_k$

* $a(i)$ -- среднее расстояние от $i$ объекта до объектов из $C_k$
* $b(i) = min_{j \neq k} b_j(i)$,  где $b_j(i)$ -- среднее расстояние от $i$ объекта до объектов из $C_j$
$$
silhouette(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$
Средний silhouette для всех точек из $\mathbf{X}$ является критерием качества кластеризации.

<center><img src='images/sil1.png'></center>

<center><img src='images/sil2.png'></center>

#### Важно!
* Эвристики и меры качества клатеризации носят лишь рекомендательный характер!
* Если они ничего не дают, то лучше ориентироваться на свои знания в предметной области
* Или "выжать" из полученной кластеризации максимум
    * *3 из 5 полученных кластеров интерпретируются - и то хорошо*

# Литература
* [Mohammed J. Zaki, Ch3](https://www.amazon.com/Data-Mining-Analysis-Fundamental-Algorithms/dp/0521766338)
* [Jure Leskovec, Anand Rajaraman, Jeffrey D. Ullman, Ch7](http://www.mmds.org/)
* [Andrew R. Webb, Keith D. Copsey, Ch11](http://eu.wiley.com/WileyCDA/WileyTitle/productCd-0470682272.html)
* [Tutorial on spectral clustering](https://arxiv.org/pdf/0711.0189.pdf)

## Вопросы?

### Пожалуйста, напишите отзыв о лекции